<a href="https://colab.research.google.com/github/LefAg/MVP-1/blob/main/MVP_Conciliacao_CT2_SE2_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP Disciplina: Sprint: Machine Learning & Analytics

Aluno: Leandro F

Análise Exploratória Contábil Conciliador — Contábil (CT2) × Fianceiro (SE2)
(Notebook com **itens numerados** e **descrições**.

O MVP automatiza a conciliação contábil entre lançamentos do CT2 e movimentos do SE2.
Ele lê os arquivos (via GitHub RAW/Colab), normaliza contas, cria visão Débito/Crédito, e aplica regras R1–R3 (NF|Código, Código|Valor, NF|Valor) para parear D×C por conta. O que não fecha vira “aberto”; nesses, o sistema busca evidências no SE2 e marca SE2_Presente.
Gera relatórios em Excel e PDF (ex.: Saldo_CT2_abertos, CT2_pairs_matches com resumo por conta, Pendências por competência) e tabelas enriquecidas com chaves, NF, Código da Parte e Fornecedor.
Por fim, um módulo de ML (RandomForest + IsolationForest) prioriza abertos com maior chance de estarem no SE2 e destaca anomalias — entregando um ML_predictions.xlsx com “Top Suspeitos” e métricas.

**Fluxo do MVP (visão de arquitetura)**

Entrada & Setup: leitura de CT2.xlsx e SE2.xlsx (GitHub RAW ou local); parâmetros (contas foco, tolerância em centavos, competência) e saída em /content/OUT.

Normalização & Enriquecimento: padroniza contas, extrai NF e Código da Parte do histórico; cria chaves combinadas e valores em centavos.

Match intra-CT2 (R1–R3): pareia D×C por Conta; gera pares e restantes (abertos).

Confirmação CT2 × SE2: nos abertos, sinaliza SE2_Presente por chaves (com tolerância).

Relatórios principais:

Saldo_CT2_abertos (Data, conta, Lote, Hist, NF, CodParte, VlrDC, SE2) + total por conta;

CT2_pairs_matches (Data, conta, NF, CodParte, valor débito/valor crédito/débito–crédito = 0) + resumo por conta;

Pendências Finais por competência.

Enriquecidos & Conferência:

CT2/SE2 enriquecidos (NF, Código, Fornecedor e chaves);

Planilhas de apoio (Visao_DC, Resumo_Conta, CT2_Matches_tol, Conferencia_CT2_vs_SE2).

ML (supervisionado + anomalias):

RandomForest prevê SE2_Presente; IsolationForest aponta anomalias nos ausentes;

Comparação com baseline (Dummy/LogReg) via CV;

Saídas: ML_predictions.xlsx (Comparação_CV, Predições, Top_Suspeitos, Matriz de Confusão, Métricas) + modelos .pkl.

**Benefícios**

Velocidade: conciliação e relatórios em minutos.

Rastreabilidade: regras claras (R1–R3), PDFs/Excels versionáveis.

Foco da equipe: priorização de pendências por risco/probabilidade (ML).

Portabilidade: roda no Colab (sem instalação local) e lê dados do GitHub.

**Como executar (resumo)**

Abrir o notebook no Google Colab.

Item 1 – Parâmetros: URLs RAW do CT2.xlsx e SE2.xlsx e outdir=/content/OUT.

Executar tudo; no final, baixar o ZIP dos relatórios.

**Próximos passos**

Enriquecer features (datas de vencimento, natureza do histórico, cluster por parceiro).

Tuning de hiperparâmetros e explicabilidade (SHAP).

Dashboard leve (Streamlit) consumindo os Excels/CSVs gerados.

Integração com ERP/filas para rodar diariamente.



## 1) Parâmetros de execução
Entradas padrão (parametrizáveis) e diretório de saída.
O que faz: define as entradas e opções do pipeline.

Principais parâmetros:

ct2_path, se2_path: caminhos dos arquivos de entrada (CT2.xlsx, SE2.xlsx).

outdir: pasta de saída (C:\MVP\MVP1\OUT por padrão).

conta_clientes, conta_fornecedores: contas foco para o relatório de abertos.

tolerancia_centavos: tolerância para “bater” valores (ex.: 1 = ± R$0,01).

competencia: rótulo de competência (ex.: 2025-09).

Saída: cria a pasta OUT se não existir e imprime um resumo dos parâmetros usados.

In [2]:

# 1) Parâmetros de execução — híbrido (Colab + VS Code) sem 'requests' (usa urllib)

import os, ssl, urllib.request
from pathlib import Path
from datetime import datetime

# Detecta Colab vs local
IS_COLAB = Path("/content").exists()

# ===== URLs RAW do GitHub (ajuste o CT2 se necessário) =====
URL_CT2 = os.environ.get("MVP_CT2_URL", "https://raw.githubusercontent.com/LefAg/MVP-1/main/CT2.xlsx")
URL_SE2 = os.environ.get("MVP_SE2_URL", "https://raw.githubusercontent.com/LefAg/MVP-1/main/SE2.xlsx")

# ===== Fallback local (opcional em VS Code) =====
ct2_local = os.environ.get("MVP_CT2_PATH", r"C:\MVP\MVP1\CT2.xlsx")
se2_local = os.environ.get("MVP_SE2_PATH", r"C:\MVP\MVP1\SE2.xlsx")

# ===== Saída =====
outdir = os.environ.get("MVP_OUTDIR", "/content/OUT" if IS_COLAB else r"C:\MVP\MVP1\OUT")
Path(outdir).mkdir(parents=True, exist_ok=True)

# ===== Pasta de dados =====
datadir = Path("/content/data" if IS_COLAB else "./data")
datadir.mkdir(parents=True, exist_ok=True)
ct2_path = datadir / "CT2.xlsx"
se2_path = datadir / "SE2.xlsx"

# Contexto SSL (evita erros em ambientes corporativos)
_SSL_CTX = ssl.create_default_context()

def _download_xlsx(url: str, dest_path: Path):
    with urllib.request.urlopen(url, context=_SSL_CTX) as resp:
        dest_path.write_bytes(resp.read())

def _ensure_file(path_target: Path, url_raw: str, local_fallback: str) -> str:
    # tenta URL RAW
    if url_raw and url_raw.startswith("http"):
        try:
            _download_xlsx(url_raw, path_target)
            print(f"OK: baixado de {url_raw} -> {path_target}")
            return str(path_target)
        except Exception as e:
            print(f"[AVISO] Falha ao baixar {url_raw}: {e}")
    # fallback local
    if local_fallback and Path(local_fallback).exists():
        print(f"Usando arquivo local: {local_fallback}")
        return local_fallback
    raise FileNotFoundError(f"Não foi possível obter o arquivo. URL={url_raw} | local={local_fallback}")

ct2_path = _ensure_file(ct2_path, URL_CT2, ct2_local)
se2_path = _ensure_file(se2_path, URL_SE2, se2_local)

# Parâmetros globais usados no restante do notebook
conta_clientes      = os.environ.get("MVP_CONTA_CLIENTES", "11030101001")
conta_fornecedores  = os.environ.get("MVP_CONTA_FORNEC",  "21010201001")
tolerancia_centavos = int(os.environ.get("MVP_TOL_CENT", "1"))
competencia         = os.environ.get("MVP_COMPETENCIA", datetime.now().strftime("%Y-%m"))

print("CT2:", ct2_path)
print("SE2:", se2_path)
print("OUT:", outdir)
print("Conta clientes:", conta_clientes, "| Conta fornecedores:", conta_fornecedores,
      "| Tolerância (centavos):", tolerancia_centavos, "| Competência:", competencia)

OK: baixado de https://raw.githubusercontent.com/LefAg/MVP-1/main/CT2.xlsx -> /content/data/CT2.xlsx
OK: baixado de https://raw.githubusercontent.com/LefAg/MVP-1/main/SE2.xlsx -> /content/data/SE2.xlsx
CT2: /content/data/CT2.xlsx
SE2: /content/data/SE2.xlsx
OUT: /content/OUT
Conta clientes: 11030101001 | Conta fornecedores: 21010201001 | Tolerância (centavos): 1 | Competência: 2025-09


## 2) Imports, normalizações e utilitários
O que faz: carrega bibliotecas e define funções auxiliares.

Funções-chave:

normalize_conta(x): limpa e padroniza o número da conta (remove pontos, traços, etc.).

extract_nf_and_codigo(hist): tenta extrair NF (ex.: “NF 12345”) e Código de Parte (heurística: 8 dígitos = PJ, 9 dígitos = PF) a partir do histórico.

to_centavos_abs(v): converte o valor monetário para centavos absolutos (int), preservando sinal separadamente.

write_pdf_table(path, title, df): gera um PDF simples com cabeçalho e até ~28 linhas do DataFrame.

as_str(series): força dtype string (pandas) e preenche NA com "" (evita erros ao concatenar).

centavos_to_str(series): transforma a coluna com centavos (int/float/NA) em string segura.

Resolver robusto de colunas (CT2/SE2): mapeia nomes “parecidos” (ignorando acentos/variações) para achar as colunas de Data, Histórico, Débito, Crédito, Valor, Lote, NF, Código.

In [4]:

# 2) Utilitários (normalização, extrações, PDF compacto)

# garante reportlab no Colab se ainda não estiver instalado
import sys, subprocess
try:
    from reportlab.pdfgen import canvas
    from reportlab.lib.pagesizes import landscape, A4
    from reportlab.lib.units import cm
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "reportlab"])
    from reportlab.pdfgen import canvas
    from reportlab.lib.pagesizes import landscape, A4
    from reportlab.lib.units import cm

import re, unicodedata
import numpy as np, pandas as pd
from collections import defaultdict
import re, unicodedata
import numpy as np, pandas as pd
from collections import defaultdict
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import landscape, A4
from reportlab.lib.units import cm
def normalize_conta(x):
    if pd.isna(x): return None
    s = str(x).strip()
    if s.endswith(".0"): s = s[:-2]
    return re.sub(r"\D", "", s)
def extract_nf_and_codigo(hist):
    if pd.isna(hist): return (None, None, None)
    s = str(hist)
    cod_match = re.search(r"\b(\d{8,9})\b", s)
    codigo = cod_match.group(1) if cod_match else None
    tipo = "PJ_8d" if (codigo and len(codigo)==8) else ("PF_9d" if (codigo and len(codigo)==9) else "OUTRO")
    nf_match = re.search(r"(?:N\.?F\.?\s*[\-:]?\s*)(\d+)", s, flags=re.IGNORECASE)
    if nf_match: nf = nf_match.group(1)
    else:
        nf_fallback = re.search(r"\b(\d{4,})\b", s)
        nf = nf_fallback.group(1) if nf_fallback else None
    return (nf, codigo, tipo)
def to_centavos_abs(v):
    if pd.isna(v): return None
    return int(round(abs(float(v))*100))
def as_str(series): return series.astype("string").fillna("")
def centavos_to_str(series): return series.apply(lambda x: "" if pd.isna(x) else str(int(x)))
DISPLAY_NAMES = {"idx_src":"idx","ValorOrig":"VlrOrig","ValorDC":"VlrDC","ValorAbsCent":"ValCent","CodigoParte":"CodParte",
                 "K_NF|CodigoParte":"Key NF|Cod","K_NF|Valor":"Key NF|Val","K_CodigoParte|Valor":"Key Cod|Val",
                 "K_NF|CodigoParte|Valor":"Key NF|Cod|Val","SE2_Presente":"SE2?","Conta_Foco":"ContaFoco","competencia":"Compet"}
def _wrap_text(s: str, max_chars: int, max_lines: int = 2):
    tokens = re.split(r'([ _\|])', s); lines, cur = [], ""
    for t in tokens:
        if len(cur)+len(t) <= max_chars: cur += t
        else:
            lines.append(cur.strip()); cur = t.strip()
            if len(lines) >= max_lines-1: break
    if cur: lines.append(cur.strip())
    if len(lines) > max_lines: lines = lines[:max_lines]
    if len("".join(tokens)) > sum(len(l) for l in lines):
        lines[-1] = (lines[-1][:max(0, max_chars-1)] + "…") if len(lines[-1]) >= max_chars else (lines[-1] + "…")
    return lines[:max_lines]
def write_pdf_table_wrapped(pdf_path, title, df, keep_columns=None, drop_hist=True, font_size=7, max_rows_pdf=None):
    df = df.copy()
    if drop_hist and "Hist" in df.columns: df = df.drop(columns=["Hist"])
    if keep_columns: df = df[[c for c in keep_columns if c in df.columns]]
    if max_rows_pdf is not None: df = df.head(int(max_rows_pdf))
    df = df.rename(columns={c: DISPLAY_NAMES.get(c,c) for c in df.columns})
    c = canvas.Canvas(pdf_path, pagesize=landscape(A4))
    width, height = landscape(A4); margin = 1.1*cm; y = height - margin
    c.setFont("Helvetica-Bold", 13); c.drawString(margin, y, str(title)); y -= 0.6*cm
    cols = list(df.columns); ncols = max(1,len(cols)); col_w = (width-2*margin)/ncols
    c.setFont("Helvetica-Bold", font_size); max_chars = max(8, int(col_w/4.0)); line_h = 0.32*cm; header_h = line_h*2
    for i, col in enumerate(cols):
        for j, ln in enumerate(_wrap_text(str(col), max_chars, 2)):
            c.drawString(margin + i*col_w, y - j*line_h, ln)
    y -= header_h + 0.1*cm
    c.setFont("Helvetica", font_size); approx = max(8, int(col_w/4.2)); row_h = 0.36*cm
    max_rows_page = int((y - margin)/ row_h)
    idx = 0; total = len(df)
    while idx < total:
        rpage = min(max_rows_page, total-idx)
        for r in range(rpage):
            row = df.iloc[idx+r]; yy = y - r*row_h
            for i,col in enumerate(cols):
                txt = "" if pd.isna(row[col]) else str(row[col])
                if len(txt) > approx: txt = txt[:approx-1]+"…"
                c.drawString(margin + i*col_w, yy, txt)
        idx += rpage
        if idx < total:
            c.showPage(); y = height - margin
            c.setFont("Helvetica-Bold", 13); c.drawString(margin, y, str(title)); y -= 0.6*cm
            c.setFont("Helvetica-Bold", font_size)
            for i, col in enumerate(cols):
                for j, ln in enumerate(_wrap_text(str(col), max_chars, 2)):
                    c.drawString(margin + i*col_w, y - j*line_h, ln)
            y -= header_h + 0.1*cm; c.setFont("Helvetica", font_size)
    c.showPage(); c.save()
def _norm(s): s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii"); s = re.sub(r"[^a-z0-9]+"," ", s.lower()).strip(); return re.sub(r"\s+"," ", s)
def _build_norm_map(df): return { _norm(c): c for c in df.columns }
def _pick(n2o, cands, contains_any=None):
    for c in cands:
        if c in n2o: return n2o[c]
    if contains_any:
        for n,orig in n2o.items():
            if any(tok in n for tok in contains_any): return orig
    return None
def resolver_ct2_cols(df):
    n = _build_norm_map(df)
    return {"data": _pick(n, ["data lcto","data lanc","dt lanc","data","dtlanc","dt"], ["data","dt"]),
            "hist": _pick(n, ["hist lanc","historico lanc","historico","hist","descricao historico"], ["hist","descr"]),
            "cta_debito": _pick(n, ["cta debito","conta debito","conta deb","debito conta","conta d","cta d","debito"], ["deb"]),
            "cta_credito": _pick(n, ["cta credito","conta credito","conta cred","credito conta","conta c","cta c","credito"], ["cred"]),
            "valor": _pick(n, ["valor lanc","vlr lanc","valor lancamento","valor","vlr","vlr lancamento"], ["valor","vlr"]),
            "lote": _pick(n, ["lote","num lote","numero lote","nr lote"], ["lote"])}
def resolver_se2_cols(df):
    n = _build_norm_map(df)
    return {"data": _pick(n, ["data mov","data","dt"], ["data","dt"]),
            "hist": _pick(n, ["historico","hist"], ["hist","descr"]),
            "valor": _pick(n, ["valor","vlr","valor mov","vlr mov"], ["valor","vlr"]),
            "nf": _pick(n, ["nf","nota fiscal","nr nf","numero nf"], None),
            "codigo": _pick(n, ["codigo","cod parceiro","codigo parte","cliente fornecedor","cod cli for","cod parte","codigo cliente","codigo fornecedor"], ["cod","cliente","fornecedor","parte"])}


## 3) Carga e mapeamento
O que faz: lê CT2.xlsx e SE2.xlsx e usa o resolver para encontrar as colunas relevantes.

Validações: se não encontrar pelo menos Valor e Histórico em CT2, para com erro orientando a ajustar o resolver (porque sem esses campos não dá para seguir).

Saída: imprime shape de cada base e o mapeamento detectado.

In [5]:

ct2 = pd.read_excel(ct2_path); se2 = pd.read_excel(se2_path)
cols_ct2 = resolver_ct2_cols(ct2); cols_se2 = resolver_se2_cols(se2)
assert cols_ct2["valor"] and cols_ct2["hist"], f"CT2 precisa de Valor/Hist. Mapeamento: {cols_ct2}"
print("CT2:", ct2.shape, "| SE2:", se2.shape)


CT2: (1596, 16) | SE2: (356, 8)


## 4) Transformação D/C (sem duplicações artificiais)
O que faz: cria a visão “movimento” com uma linha D (valor positivo) e uma linha C (valor negativo) por lançamento, usando as contas de débito e crédito da CT2.

Campos gerados:
Conta, DC (“D” ou “C”), ValorDC (+ para D; − para C), além de Data, Lote, Hist e ValorOrig.

Validação de integridade: checa se a soma da visão D/C bate com a soma da CT2 original (garante que não criamos/remoções indevidas de valor).

In [6]:

ct2["ContaDeb"] = ct2[cols_ct2["cta_debito"]].apply(normalize_conta) if cols_ct2["cta_debito"] else None
ct2["ContaCre"] = ct2[cols_ct2["cta_credito"]].apply(normalize_conta) if cols_ct2["cta_credito"] else None
ct2["ValorOrig"] = pd.to_numeric(ct2[cols_ct2["valor"]], errors="coerce")
ct2["HistOrig"]  = as_str(ct2[cols_ct2["hist"]])
rows=[]
for idx,r in ct2.iterrows():
    base={"idx_src":idx,"Data": r.get(cols_ct2["data"]) if cols_ct2["data"] else r.get(ct2.columns[0]),
          "Lote": r.get("Lote", r.get(cols_ct2["lote"])) if cols_ct2["lote"] else r.get("Lote"),
          "Hist": r.get("HistOrig"),"ValorOrig": r.get("ValorOrig")}
    if cols_ct2["cta_debito"] and pd.notna(r["ContaDeb"]):
        rr=base.copy(); rr["Conta"]=r["ContaDeb"]; rr["DC"]="D"; rr["ValorDC"]=abs(r["ValorOrig"]); rows.append(rr)
    if cols_ct2["cta_credito"] and pd.notna(r["ContaCre"]):
        rr=base.copy(); rr["Conta"]=r["ContaCre"]; rr["DC"]="C"; rr["ValorDC"]=-abs(r["ValorOrig"]); rows.append(rr)
visao_dc=pd.DataFrame(rows)
print("Linhas DC:", len(visao_dc))


Linhas DC: 1809


## 5) Normalizações/Enriquecimento (CT2) + Chaves
O que faz:

Extrai NF, Código da Parte (e o tipo: PJ_8d, PF_9d, OUTRO) a partir do histórico da CT2.

Cria o ValorAbsCent (valor absoluto em centavos, inteiro) para as chaves.

Constrói as chaves de conciliação:

K_NF|CodigoParte = NF + "|" + Codigo

K_NF|Valor = NF + "|" + ValorAbsCent

K_CodigoParte|Valor = Codigo + "|" + ValorAbsCent

K_NF|CodigoParte|Valor = NF + "|" + Codigo + "|" + ValorAbsCent

Resumo auxiliar: gera resumo_conta (Débitos, Créditos, Saldo, Qtd) por Conta × DC para conferência.

In [7]:

nf,codigo,tipo=[],[],[]
for s in visao_dc["Hist"].fillna(""):
    n,c,t=extract_nf_and_codigo(s); nf.append(n); codigo.append(c); tipo.append(t)
visao_dc["NF"]=as_str(pd.Series(nf))
visao_dc["CodigoParte"]=as_str(pd.Series(codigo))
visao_dc["TipoParte"]=as_str(pd.Series(tipo))
visao_dc["ValorAbsCent"]=visao_dc["ValorDC"].apply(to_centavos_abs)
val_ct2_str=centavos_to_str(visao_dc["ValorAbsCent"])
nf_ct2_str=as_str(visao_dc["NF"]); cod_ct2_str=as_str(visao_dc["CodigoParte"])
visao_dc["K_NF|CodigoParte"]=nf_ct2_str+"|"+cod_ct2_str
visao_dc["K_NF|Valor"]=nf_ct2_str+"|"+val_ct2_str
visao_dc["K_CodigoParte|Valor"]=cod_ct2_str+"|"+val_ct2_str
visao_dc["K_NF|CodigoParte|Valor"]=nf_ct2_str+"|"+cod_ct2_str+"|"+val_ct2_str


## 6) Transformação SE2 + chaves (valor absoluto)
O que faz:

Padroniza o histórico (Hist2) e tenta obter NF e Código:

Se a SE2 já tiver colunas de NF/Código mapeadas, usa-as.

Caso contrário, extrai do Hist2 com a mesma heurística da CT2.

Converte Valor para número, cria ValorAbsCent.

Gera as mesmas chaves usadas na CT2 (compatibilidade perfeita para comparar).

In [8]:

se2=se2.copy()
se2["Hist2"]=as_str(se2[cols_se2["hist"]]) if cols_se2["hist"] else as_str(se2.iloc[:,0])
se2["NF"]=as_str(se2[cols_se2["nf"]]) if cols_se2["nf"] else as_str(pd.Series([extract_nf_and_codigo(s)[0] for s in se2["Hist2"].fillna("")]))
se2["Codigo"]=as_str(se2[cols_se2["codigo"]]) if cols_se2["codigo"] else as_str(pd.Series([extract_nf_and_codigo(s)[1] for s in se2["Hist2"].fillna("")]))
se2["Valor"]=pd.to_numeric(se2[cols_se2["valor"]], errors="coerce") if cols_se2["valor"] else pd.to_numeric(se2.iloc[:,1], errors="coerce")
se2["ValorAbsCent"]=se2["Valor"].apply(to_centavos_abs)
nf_str, cod_str, val_str = as_str(se2["NF"]), as_str(se2["Codigo"]), centavos_to_str(se2["ValorAbsCent"])
se2["K_NF|CodigoParte"]=nf_str+"|"+cod_str
se2["K_NF|Valor"]=nf_str+"|"+val_str
se2["K_CodigoParte|Valor"]=cod_str+"|"+val_str
se2["K_NF|CodigoParte|Valor"]=nf_str+"|"+cod_str+"|"+val_str


## 6.1) CT2 & SE2 — enriquecidos (export)

In [10]:
# 6.1) CT2 & SE2 — enriquecidos (export)  [VERSÃO CORRIGIDA]
# Gera CT2/SE2 com chaves + NF + CodigoParte + Fornecedor, normalizando datas para ordenar.

# garante xlsxwriter no Colab/ambiente atual
import sys, subprocess
try:
    import xlsxwriter  # noqa: F401
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xlsxwriter", "openpyxl"])
    import xlsxwriter
import numpy as np
import pandas as pd
from pathlib import Path

def _to_dt(series):
    """Converte para datetime (NaT quando não converter)."""
    return pd.to_datetime(series, errors="coerce")

# ------- CT2 enriquecido -------
ct2_enriq = visao_dc.copy()

# Fornecedor no CT2: marca o código apenas quando a conta é a de fornecedores
ct2_enriq["Fornecedor"] = np.where(
    ct2_enriq["Conta"] == conta_fornecedores,
    ct2_enriq["CodigoParte"].fillna(""),
    ""
)

# Normaliza Data para ordenar sem conflito de tipos
ct2_enriq["DataSort"] = _to_dt(ct2_enriq["Data"])

cols_ct2_enriq = [
    "Data","Conta","DC","ValorDC",
    "NF","CodigoParte","Fornecedor",
    "K_NF|CodigoParte","K_CodigoParte|Valor","K_NF|Valor","K_NF|CodigoParte|Valor"
]
cols_ct2_enriq = [c for c in cols_ct2_enriq if c in ct2_enriq.columns]

ct2_enriq = (
    ct2_enriq
      .sort_values(["Conta","DataSort"], kind="stable")
      .drop(columns=["DataSort"])
      [cols_ct2_enriq]
)

# ------- SE2 enriquecido -------
# Pega a coluna de data mapeada (se existir) e normaliza
se2_data_col = cols_se2.get("data")
se2_data_raw = se2[se2_data_col] if se2_data_col else pd.Series([pd.NaT]*len(se2), name="Data")
se2_data_sort = _to_dt(se2_data_raw)

se2_enriq = pd.DataFrame({
    "Data": se2_data_raw,
    "NF": se2["NF"],
    "CodigoParte": se2["Codigo"],
    "Fornecedor": se2["Codigo"],        # sem nome, usamos o identificador do parceiro
    "Valor": se2["Valor"],
    "ValorAbsCent": se2["ValorAbsCent"],
    "K_NF|CodigoParte": se2["K_NF|CodigoParte"],
    "K_CodigoParte|Valor": se2["K_CodigoParte|Valor"],
    "K_NF|Valor": se2["K_NF|Valor"],
    "K_NF|CodigoParte|Valor": se2["K_NF|CodigoParte|Valor"],
})
se2_enriq["DataSort"] = se2_data_sort

se2_enriq = (
    se2_enriq
      .sort_values(["CodigoParte","DataSort"], kind="stable")
      .drop(columns=["DataSort"])
)

# ------- Exporta Excel (duas abas) -------
out_base = Path(outdir)
with pd.ExcelWriter(out_base / "CT2_SE2_enriquecidos.xlsx", engine="xlsxwriter") as wr:
    ct2_enriq.to_excel(wr, index=False, sheet_name="CT2_enriquecido")
    se2_enriq.to_excel(wr, index=False, sheet_name="SE2_enriquecido")

# ------- Exporta PDFs compactos -------
write_pdf_table_wrapped(
    str(out_base / "CT2_enriquecido.pdf"),
    "CT2 — enriquecido (chaves + NF/CodigoParte/Fornecedor)",
    ct2_enriq,
    keep_columns=cols_ct2_enriq,
    drop_hist=True,
    font_size=7
)

write_pdf_table_wrapped(
    str(out_base / "SE2_enriquecido.pdf"),
    "SE2 — enriquecido (chaves + NF/CodigoParte/Fornecedor)",
    se2_enriq,
    keep_columns=[c for c in se2_enriq.columns],
    drop_hist=True,
    font_size=7
)

print("CT2 & SE2 enriquecidos exportados (datas normalizadas).")

CT2 & SE2 enriquecidos exportados (datas normalizadas).


## 7) Match intra-CT2 (R1/R2/R3) + export PDF/Excel de pares
O que faz: tenta parear débitos e créditos dentro da própria CT2 (mesma conta) por camadas de regras:

R1: K_NF|CodigoParte

R2: K_CodigoParte|Valor (com tolerância: procura valores ± tolerância em centavos)

R3: K_NF|Valor (com tolerância)

Lógica: agrupa por Conta + chave + sinal (D/C) e vai emparelhando 1 a 1. Quando encontra par, remove os índices do “pool” aberto e continua.

Saídas:

ct2_pairs: tabela com idx_D, idx_C, Conta, Regra, Diff_cent (diferença entre valores em centavos).

ct2_restantes: sobras da CT2 ainda em aberto (não pareadas internamente).

Arquivos: CT2_pairs_matches.xlsx e CT2_pairs_matches.pdf (amostra das primeiras linhas).

In [11]:

tol=int(tolerancia_centavos)
def match_in_layers(df):
    df=df.copy(); df["sign"]=np.where(df["ValorDC"]>=0,1,-1)
    open_idx=df.index.tolist(); pairs=[]
    def try_match(key_cols, rule_name):
        nonlocal open_idx,pairs
        grp=df.loc[open_idx].groupby(["Conta"]+key_cols+["sign"])
        from collections import defaultdict
        buckets=defaultdict(lambda:{1:[], -1:[]})
        for ix,_ in grp:
            sign=ix[-1]; key=tuple(ix[1:-1])
            for i in grp.indices[ix]: buckets[key][sign].append(i)
        new=[]
        for key,d in buckets.items():
            pos,neg=d[1],d[-1]
            while pos and neg:
                i=pos.pop(); j=neg.pop()
                vi=df.at[i,"ValorAbsCent"]; vj=df.at[j,"ValorAbsCent"]
                diff=None if (pd.isna(vi) or pd.isna(vj)) else abs(int(vi)-int(vj))
                if diff is None or diff<=tol: new.append((i,j,rule_name,diff))
        flat=set([i for x in new for i in x[:2]])
        open_idx=[i for i in open_idx if i not in flat]; pairs.extend(new)
    try_match(["K_NF|CodigoParte"],"R1"); try_match(["K_CodigoParte|Valor"],"R2"); try_match(["K_NF|Valor"],"R3")
    paired_rows=[]
    for i,j,rule,diff in pairs:
        d_idx=i if df.at[i,"DC"]=="D" else j; c_idx=j if df.at[i,"DC"]=="D" else i
        paired_rows.append({"idx_D":d_idx,"idx_C":c_idx,"Conta":df.at[i,"Conta"],"Regra":rule,"Diff_cent":diff})
    return pd.DataFrame(paired_rows), df.loc[open_idx].copy()
ct2_pairs, ct2_restantes = match_in_layers(visao_dc)


## 8) CT2 abertos x correspondência SE2 (export dos COM correspondência)
O que faz: pega os abertos (restantes) da CT2 nas contas foco (Clientes/Fornecedores) e marca se existe correspondência na SE2 pelas mesmas regras (R1/R2/R3).

Tolerância: para as chaves que incluem Valor, procura variações ± tolerancia_centavos.

Saídas:

abertos_ct2_conf: CT2 abertos com flags R1, R2, R3 e SE2_Presente (True se qualquer regra bater).

abertos_com_se2: somente os que têm correspondência (para auditoria do que “bateu”).

Arquivos: CT2_abertos_com_correspondencia_SE2.xlsx e CT2_abertos_com_correspondencia_SE2.pdf.

In [12]:

contas_foco={conta_clientes, conta_fornecedores}
abertos_ct2=ct2_restantes[ct2_restantes["Conta"].isin(contas_foco)].copy()
def lookup_match_flags(df_ct2, df_se2):
    keys=[("K_NF|CodigoParte","R1"),("K_CodigoParte|Valor","R2"),("K_NF|Valor","R3")]
    tol=int(tolerancia_centavos); flags={k:[] for _,k in keys}; any_flag=[]
    for _, r in df_ct2.iterrows():
        has_any=False
        for key_col, rule in keys:
            k=r.get(key_col,""); found=False
            if pd.notna(k) and k!="":
                if "Valor" in key_col:
                    base=r.get("ValorAbsCent")
                    if pd.isna(base): mask=(df_se2[key_col]==k)
                    else:
                        candidates=[str(int(base+d)) for d in range(-tol,tol+1)]
                        left=str(k).split("|")[0]; keys_cand=[f"{left}|{c}" for c in candidates]
                        mask=df_se2[key_col].isin(keys_cand)
                else: mask=(df_se2[key_col]==k)
                found=bool(mask.any())
            flags[rule].append(found); has_any=has_any or found
        any_flag.append(has_any)
    df_flags=pd.DataFrame(flags); df_flags["SE2_Presente"]=any_flag; return df_flags
abertos_flags=lookup_match_flags(abertos_ct2, se2)
abertos_ct2_conf=pd.concat([abertos_ct2.reset_index(drop=True), abertos_flags], axis=1)
abertos_com_se2=abertos_ct2_conf[abertos_ct2_conf["SE2_Presente"]].copy()


## 9) Saldo CT2 — Abertos (Excel + PDF)
O que faz: relatório simples dos abertos (após match) com coluna SE2 (True/False).

Colunas: Data, Lote, Hist, NF, CodigoParte, ValorDC, SE2.

Arquivos: Saldo_CT2_abertos.xlsx e Saldo_CT2_abertos.pdf (até 200 linhas no PDF para visual rápido).

In [13]:

# 9) Saldo CT2 — Abertos (layout + TOTAL por conta dentro do mesmo arquivo, com 2 casas decimais)

import pandas as pd
from pathlib import Path

assert 'abertos_ct2_conf' in globals(), "Execute os itens anteriores."

# ---- base e layout exigido ----
abertos_ct2_simple = abertos_ct2_conf.copy()
abertos_ct2_simple["SE2"] = abertos_ct2_simple["SE2_Presente"].fillna(False)

df_out = (
    abertos_ct2_simple.rename(columns={
        "Conta":"conta",
        "CodigoParte":"CodParte",
        "ValorDC":"VlrDC"
    })[["Data","conta","Lote","Hist","NF","CodParte","VlrDC","SE2"]]
     .sort_values(["conta","Data"], kind="stable")
     .copy()
)

# ---- totais por conta (para anexar no final do relatório) ----
totais = (
    df_out.groupby("conta", dropna=False)["VlrDC"]
          .sum()
          .reset_index()
          .rename(columns={"VlrDC":"Total_VlrDC"})
          .sort_values("conta", kind="stable")
)

total_rows = (
    totais.assign(Data="", Lote="", Hist="", NF="", CodParte="", SE2="")
          [["Data","conta","Lote","Hist","NF","CodParte","Total_VlrDC","SE2"]]
          .rename(columns={"Total_VlrDC":"VlrDC"})
)

df_out_com_totais = pd.concat([df_out, total_rows], ignore_index=True)

# ---- gravação em Excel COM formatação numérica (duas casas + milhar) ----
out_path = Path(outdir) / "Saldo_CT2_abertos.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as wr:
    # aba principal
    df_out_com_totais.to_excel(wr, index=False, sheet_name="Saldo_CT2_abertos")
    ws1 = wr.sheets["Saldo_CT2_abertos"]

    # aba de resumo dentro do MESMO arquivo (nome solicitado)
    resumo_nome = "Saldo_CT2_abertos_resumo"
    totais.to_excel(wr, index=False, sheet_name=resumo_nome)
    ws2 = wr.sheets[resumo_nome]

    # formato com 2 casas decimais; Excel adaptará separadores ao locale (ex.: 1.000,00 em PT-BR)
    wb = wr.book
    fmt_2c = wb.add_format({"num_format": "#,##0.00"})

    # aplica ao VlrDC da aba principal (coluna G = índice 6) e à Total_VlrDC da aba de resumo (coluna B = índice 1)
    # (se mudar ordem das colunas no futuro, calculamos o índice dinamicamente)
    def _col_letter(i):  # 0->A, 1->B...
        letters = ""
        i += 1
        while i:
            i, r = divmod(i-1, 26)
            letters = chr(65+r) + letters
        return letters

    col_vlrdc_idx = df_out_com_totais.columns.get_loc("VlrDC")
    col_total_idx = totais.columns.get_loc("Total_VlrDC")

    ws1.set_column(f"{_col_letter(col_vlrdc_idx)}:{_col_letter(col_vlrdc_idx)}", 14, fmt_2c)
    ws2.set_column(f"{_col_letter(col_total_idx)}:{_col_letter(col_total_idx)}", 14, fmt_2c)

# ---- PDF principal (com totais ao final) ----
write_pdf_table_wrapped(
    str(Path(outdir) / "Saldo_CT2_abertos.pdf"),
    "Saldo CT2 — Abertos (com total por conta ao final)",
    df_out_com_totais,
    keep_columns=["Data","conta","Lote","Hist","NF","CodParte","VlrDC","SE2"],
    drop_hist=False,
    font_size=7
)

print("Saldo_CT2_abertos.xlsx salvo com aba de RESUMO e valores formatados (2 casas decimais).")

Saldo_CT2_abertos.xlsx salvo com aba de RESUMO e valores formatados (2 casas decimais).


## 10) CT2 PARES — novo formato (linhas + resumo)

In [14]:

# 10) CT2 PARES — layout solicitado (Data, conta, NF, CodParte, valor debito, valor credito, debito - credito = 0)

import numpy as np
import pandas as pd
from pathlib import Path

assert 'ct2_pairs' in globals() and 'visao_dc' in globals(), "Execute os itens anteriores."

def _first_non_empty(a, b):
    a = "" if pd.isna(a) else str(a)
    b = "" if pd.isna(b) else str(b)
    return a if a.strip() != "" else b

rows = []
for _, par in ct2_pairs.iterrows():
    d = visao_dc.loc[par["idx_D"]]
    c = visao_dc.loc[par["idx_C"]]

    # Data: menor entre Débito e Crédito (fallback para texto original se NaT)
    d_dt = pd.to_datetime(d["Data"], errors="coerce")
    c_dt = pd.to_datetime(c["Data"], errors="coerce")
    if pd.notna(d_dt) or pd.notna(c_dt):
        data = min([x for x in [d_dt, c_dt] if pd.notna(x)]).date().isoformat()
    else:
        data = str(d["Data"])  # fallback

    conta = _first_non_empty(d.get("Conta"), c.get("Conta"))
    nf    = _first_non_empty(d.get("NF"), c.get("NF"))
    cod   = _first_non_empty(d.get("CodigoParte"), c.get("CodigoParte"))

    # Valores absolutos
    val_deb  = abs(float(d["ValorDC"])) if d["DC"] == "D" else abs(float(c["ValorDC"]))
    val_cred = abs(float(c["ValorDC"])) if c["DC"] == "C" else abs(float(d["ValorDC"]))

    # Diferença (em moeda) e checagem por centavos dentro da tolerância
    dif_moeda = round(val_deb - val_cred, 2)
    dif_cent  = abs(int(round(val_deb*100)) - int(round(val_cred*100)))

    # Se dentro da tolerância de centavos, força visual = 0.00
    if dif_cent <= int(tolerancia_centavos):
        dif_moeda = 0.00

    rows.append({
        "Data": data,
        "conta": conta,
        "NF": nf,
        "CodParte": cod,
        "valor debito": round(val_deb, 2),
        "valor credito": round(val_cred, 2),
        "debito - credito": round(dif_moeda, 2),
    })

pares_layout = pd.DataFrame(rows).sort_values(["conta", "Data"], kind="stable")

# --- Resumo por conta (tem que zerar)
resumo_pares = (
    pares_layout.groupby("conta", dropna=False)[["valor debito","valor credito","debito - credito"]]
               .agg(qtd=("valor debito","count"),
                    valor_debito=("valor debito","sum"),
                    valor_credito=("valor credito","sum"),
                    saldo=("debito - credito","sum"))
               .reset_index()
               .loc[:, ["conta","qtd","valor_debito","valor_credito","saldo"]]
               .sort_values("conta", kind="stable")
)

# --- Exporta Excel (duas abas)
with pd.ExcelWriter(Path(outdir) / "CT2_pairs_matches.xlsx", engine="xlsxwriter") as wr:
    pares_layout.to_excel(wr, index=False, sheet_name="CT2_pairs_matches")
    resumo_pares.to_excel(wr, index=False, sheet_name="Resumo")

# --- Exporta PDFs (linhas + resumo)
write_pdf_table_wrapped(
    str(Path(outdir) / "CT2_pairs_matches.pdf"),
    "CT2 PARES — Linhas (por conta)",
    pares_layout,
    keep_columns=["Data","conta","NF","CodParte","valor debito","valor credito","debito - credito"],
    drop_hist=True,
    font_size=7
)

write_pdf_table_wrapped(
    str(Path(outdir) / "CT2_pairs_matches_resumo.pdf"),
    "CT2 PARES — Resumo por conta",
    resumo_pares,
    keep_columns=["conta","qtd","valor_debito","valor_credito","saldo"],
    drop_hist=True,
    font_size=8
)

## 11) Pendências Finais + README
 que faz: gera as pendências = CT2 abertos (contas foco) que não têm presença na SE2 (SE2_Presente == False).

Campos extras:

competencia: derivada de Data (AAAA-MM) para facilitar filtros.

Obs: coluna vazia com validação de dados (lista suspensa no Excel) para classificar o motivo (ex.: “Falta provisão”, “Duplicidade”, etc.).

Conta_Foco: rotula “Clientes” / “Fornecedores” / “Outras”.

Arquivos:

MVP_pendencias_finais_v2.xlsx com duas abas: Pendencias_Finais e Pendencias_<competencia>.

Pendencias_Finais_<competencia>.pdf (se não houver registros na competência, cai para um PDF “fallback” com geral).

In [15]:

pendencias = abertos_ct2_conf[~abertos_ct2_conf["SE2_Presente"]].copy()
pendencias["competencia"] = pd.to_datetime(pendencias["Data"], errors="coerce").dt.strftime("%Y-%m")
pendencias["Obs"] = ""
pendencias["Conta_Foco"] = np.where(pendencias["Conta"]==conta_clientes,"Clientes",
                             np.where(pendencias["Conta"]==conta_fornecedores,"Fornecedores","Outras"))
with pd.ExcelWriter(Path(outdir)/"MVP_pendencias_finais_v2.xlsx", engine="xlsxwriter") as writer:
    pendencias.to_excel(writer, index=False, sheet_name="Pendencias_Finais")
    (pendencias[pendencias["competencia"]==competencia]).to_excel(writer, index=False, sheet_name=f"Pendencias_{competencia}")
df_pdf = pendencias[pendencias["competencia"]==competencia]
title = f"Pendências Finais - {competencia}" if len(df_pdf)>0 else "Pendências Finais - Todas as competências"
src   = df_pdf if len(df_pdf)>0 else pendencias
cols_pend = ["Data","NF","CodigoParte","Conta","DC","ValorDC","R1","R2","R3","SE2_Presente","competencia","Obs","Conta_Foco"]
write_pdf_table_wrapped(str(Path(outdir)/f"Pendencias_Finais_{competencia}.pdf"), title, src,
                        keep_columns=cols_pend, drop_hist=True, font_size=7)
with open(Path(outdir).parent / "README_EXECUCAO.txt","w",encoding="utf-8") as f:
    f.write(f"README v5 gerado em {datetime.now().isoformat()}\nOUT: {outdir}\nTol(centavos): {tolerancia_centavos}\nComp: {competencia}\n")


## 12) Exports auxiliares de conferência
O que faz: produz duas planilhas de apoio:

CT2_conciliacao_enriquecido.xlsx

Visao_DC (todas as linhas D/C enriquecidas)

Resumo_Conta (conferência de totais por conta e DC)

MVP_CT2_intra_matches_e_conferencia.xlsx

CT2_Matches_tol (pares D×C achados)

CT2_Abertos (o que sobrou da CT2)

Conferencia_CT2_vs_SE2 (abertos com flags de presença na SE2)

In [16]:
# 11A) Exports auxiliares de conferência
# O que faz: cria dois arquivos de apoio:
#   1) CT2_conciliacao_enriquecido.xlsx  -> Visao_DC + Resumo_Conta
#   2) MVP_CT2_intra_matches_e_conferencia.xlsx -> CT2_Matches_tol + CT2_Abertos + Conferencia_CT2_vs_SE2
#
# Requisitos: variáveis visao_dc, ct2_pairs, ct2_restantes, abertos_ct2_conf já criadas nos itens anteriores.

import pandas as pd
import numpy as np
from pathlib import Path

assert 'visao_dc' in globals(), "Execute a geração da visão D/C."
assert 'ct2_pairs' in globals(), "Execute o pareamento intra-CT2."
assert 'ct2_restantes' in globals(), "Execute o pareamento intra-CT2."
assert 'abertos_ct2_conf' in globals(), "Execute a confirmação CT2 x SE2 (flags)."

# -------- 1) CT2_conciliacao_enriquecido.xlsx --------
# Visao_DC (todas as linhas D/C enriquecidas)
visao_dc_export = visao_dc.copy()

# Resumo_Conta (conferência por Conta e DC)
resumo_conta = (
    visao_dc_export
      .groupby(["Conta","DC"], dropna=False)["ValorDC"]
      .agg(qtd="count", soma="sum", soma_abs=lambda s: s.abs().sum())
      .reset_index()
      .sort_values(["Conta","DC"], kind="stable")
)

out1 = Path(outdir) / "CT2_conciliacao_enriquecido.xlsx"
with pd.ExcelWriter(out1, engine="xlsxwriter") as wr:
    visao_dc_export.to_excel(wr, index=False, sheet_name="Visao_DC")
    resumo_conta.to_excel(wr, index=False, sheet_name="Resumo_Conta")

    # formata colunas numéricas com 2 casas no resumo
    wb = wr.book
    fmt2 = wb.add_format({"num_format": "#,##0.00"})
    ws_r = wr.sheets["Resumo_Conta"]
    for col_name in ["soma","soma_abs"]:
        if col_name in resumo_conta.columns:
            col_idx = resumo_conta.columns.get_loc(col_name)
            # aplica a coluna inteira
            ws_r.set_column(col_idx, col_idx, 14, fmt2)

# -------- 2) MVP_CT2_intra_matches_e_conferencia.xlsx --------
# CT2_Matches_tol (pares D×C achados)
ct2_matches_tol = ct2_pairs.copy()  # mantém: idx_D, idx_C, Conta, Regra, Diff_cent

# CT2_Abertos (o que sobrou da CT2 após pareamento)
ct2_abertos = ct2_restantes.copy()

# Conferencia_CT2_vs_SE2 (abertos com flags de presença na SE2)
conferencia = abertos_ct2_conf.copy()

out2 = Path(outdir) / "MVP_CT2_intra_matches_e_conferencia.xlsx"
with pd.ExcelWriter(out2, engine="xlsxwriter") as wr:
    ct2_matches_tol.to_excel(wr, index=False, sheet_name="CT2_Matches_tol")
    ct2_abertos.to_excel(wr, index=False, sheet_name="CT2_Abertos")
    conferencia.to_excel(wr, index=False, sheet_name="Conferencia_CT2_vs_SE2")

print("Arquivos de conferência gerados:")
print(" -", out1)
print(" -", out2)

Arquivos de conferência gerados:
 - /content/OUT/CT2_conciliacao_enriquecido.xlsx
 - /content/OUT/MVP_CT2_intra_matches_e_conferencia.xlsx


## 13) ML (supervisionado + anomalias)
Objetivo: auxiliar priorização/triagem do que provavelmente tem (ou não tem) correspondência na SE2.

Rótulo: SE2_Presente (1/0) gerado na etapa 8.

Features:

Numéricas: valor_abs, nf_len, cod_len, hist_len, ano, mes, dia.

Categóricas: conta_cat, dc_cat (OneHotEncoder aplicado em pipeline).

Sinais auxiliares: is_debito, is_credito.

Modelo: RandomForestClassifier(class_weight="balanced").

Treina/testa (split 75/25) somente se houver variedade suficiente de rótulos e amostra mínima.

Calcula métricas: Accuracy, F1, ROC_AUC + Matriz de Confusão.

Anomalias (IsolationForest):

Rodado apenas sobre os casos onde SE2_Presente == 0 (não bateu) — para destacar outliers.

Scores e prioridade:

rf_score (probabilidade do modelo), iso_score (quando disponível),

Prioridade: Alta/Média/Baixa (regras simples combinando RF e anomalia).

Saídas:

ML_predictions.xlsx com abas:

Predicoes_Todas (dataset com scores),

Top_Suspeitos (não-batidos priorizados),

Matriz_Confusao, Metricas.

Artefatos: ML_ct2_se2_classifier.pkl, ML_ct2_isolation_forest.pkl (se treinado), ML_feature_columns.json.

In [17]:
# 13) ML (supervisionado + anomalias) — BLOCO COMPLETO E ENXUTO

# O que faz:
#  - 13.0) Pré-ML: fixa seed, monta dataset de features, roda baseline/comparação (CV) entre Dummy, LogReg e RandomForest.
#  - 13.1) Treino hold-out com RandomForest e detecção de anomalias (IsolationForest) nos "ausentes".
#  - 13.2) Geração de outputs: ML_predictions.xlsx (abas Comparacao_CV, Predicoes_Todas, Top_Suspeitos, Matriz_Confusao, Metricas)
#           + modelos .pkl e lista de features em OUT.
# Requisitos:
#  - Variável `abertos_ct2_conf` (do item CT2 × SE2) já carregada
#  - Variável `outdir` definida (se não, cai no fallback "./OUT")

import os, random, json
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from joblib import dump

# ---------- 13.0) Pré-ML: seed + dataset + baseline/comparação (CV) ----------
SEED = 42
random.seed(SEED); np.random.seed(SEED); os.environ["PYTHONHASHSEED"] = str(SEED)

assert 'abertos_ct2_conf' in globals(), "Execute o item CT2×SE2 antes do ML (precisa de 'abertos_ct2_conf')."

# Função util local (se não veio de itens anteriores)
def _as_str(series):
    return series.astype("string").fillna("")

# Monta dataset de ML (se já existir, reaproveita)
if 'data_ml' not in globals():
    data_ml = abertos_ct2_conf.copy()
    data_ml["is_debito"]  = (data_ml["DC"]=="D").astype(int)
    data_ml["is_credito"] = (data_ml["DC"]=="C").astype(int)
    data_ml["valor_abs"]  = data_ml["ValorDC"].abs().fillna(0.0)
    data_ml["nf_len"]     = _as_str(data_ml["NF"]).apply(len)
    data_ml["cod_len"]    = _as_str(data_ml["CodigoParte"]).apply(len)
    data_ml["hist_len"]   = _as_str(data_ml["Hist"]).apply(len)
    data_ml["ano"]        = pd.to_datetime(data_ml["Data"], errors="coerce").dt.year.fillna(0).astype(int)
    data_ml["mes"]        = pd.to_datetime(data_ml["Data"], errors="coerce").dt.month.fillna(0).astype(int)
    data_ml["dia"]        = pd.to_datetime(data_ml["Data"], errors="coerce").dt.day.fillna(0).astype(int)
    data_ml["conta_cat"]  = _as_str(data_ml["Conta"])
    data_ml["dc_cat"]     = _as_str(data_ml["DC"])

if 'features' not in globals():
    features = ["valor_abs","is_debito","is_credito","nf_len","cod_len","hist_len",
                "ano","mes","dia","conta_cat","dc_cat"]

target   = "SE2_Presente"
cat_cols = ["conta_cat","dc_cat"]

X = data_ml[features].copy()
y = data_ml[target].astype(int)

pre = ColumnTransformer([("onehot", OneHotEncoder(handle_unknown="ignore"), cat_cols)],
                        remainder="passthrough")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Baseline e comparação via CV
modelos = {
    "Dummy(stratified)": DummyClassifier(strategy="stratified", random_state=SEED),
    "LogReg": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=SEED)
}

resultados = []
if y.nunique() >= 2 and len(X) >= 10:
    for nome, est in modelos.items():
        pipe = Pipeline([("pre", pre), ("est", est)])
        f1  = cross_val_score(pipe, X, y, cv=cv, scoring="f1").mean()
        acc = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy").mean()
        resultados.append({"modelo": nome, "F1_cv": round(f1,4), "ACC_cv": round(acc,4)})
else:
    resultados.append({"modelo": "Dados insuficientes", "F1_cv": np.nan, "ACC_cv": np.nan})

df_resultados = pd.DataFrame(resultados).sort_values("F1_cv", ascending=False)

# ---------- 13.1) Treino hold-out (RF) + anomalias (IsolationForest) ----------
clf = Pipeline([("pre", pre),
                ("rf", RandomForestClassifier(class_weight="balanced", n_estimators=200, random_state=SEED))])

trained = False
acc=f1=auc=np.nan
y_test=y_pred=None

if y.nunique()>1 and len(X)>20:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=SEED, stratify=y
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:,1]
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)
    try:
        auc = roc_auc_score(y_test, y_prob)
    except Exception:
        auc = float("nan")
    print("RF (hold-out) -> ACC:", round(acc,4), "| F1:", round(f1,4), "| ROC_AUC:", round(auc if pd.notna(auc) else 0,4))
    trained = True
else:
    print("ML: dados insuficientes para hold-out (rótulo único ou amostra pequena).")

# ---------- 13.2) Outputs: Excel + modelos ----------
if 'outdir' not in globals() or not outdir:
    outdir = "./OUT"
Path(outdir).mkdir(parents=True, exist_ok=True)

if trained:
    # IsolationForest apenas nos ausentes (negativos)
    data_ml["iso_score"] = np.nan
    X_no = data_ml.loc[data_ml[target]==0, features].copy()
    if len(X_no) >= 10:
        iso_pre = ColumnTransformer([("onehot", OneHotEncoder(handle_unknown="ignore"), cat_cols)],
                                    remainder="passthrough")
        X_no_enc = iso_pre.fit_transform(X_no)
        iso = IsolationForest(random_state=SEED, contamination=0.10)
        iso.fit(X_no_enc)
        iso_scores = -iso.decision_function(X_no_enc)
        data_ml.loc[data_ml[target]==0, "iso_score"] = iso_scores

    # Probabilidades para todas as linhas
    data_ml["rf_score"] = clf.predict_proba(X)[:,1]

    # Priorização
    def prior(row):
        s = row.get("rf_score", 0.0)
        if row[target]==1:  # já confirmado na SE2 -> baixa prioridade
            return "Baixa"
        if pd.notna(row.get("iso_score")) and row["iso_score"] > np.nanpercentile(data_ml["iso_score"], 85):
            return "Alta"
        if s>=0.66: return "Alta"
        if s>=0.33: return "Média"
        return "Baixa"
    data_ml["Prioridade"] = data_ml.apply(prior, axis=1)

    # Salva modelos e metadados
    dump(clf, str(Path(outdir)/"ML_ct2_se2_classifier.pkl"))
    if 'iso' in locals():
        dump(iso, str(Path(outdir)/"ML_ct2_isolation_forest.pkl"))
    with open(Path(outdir)/"ML_feature_columns.json","w",encoding="utf-8") as f:
        json.dump(features, f, ensure_ascii=False, indent=2)

    # Excel consolidado
    pred_all = data_ml.copy().sort_values(["Prioridade","rf_score"], ascending=[True, False])
    top_suspeitos = pred_all[pred_all[target]==0].head(200)

    with pd.ExcelWriter(str(Path(outdir)/"ML_predictions.xlsx"), engine="xlsxwriter") as wr:
        # Comparação CV
        df_resultados.to_excel(wr, index=False, sheet_name="Comparacao_CV")
        # Predições
        pred_all.to_excel(wr, index=False, sheet_name="Predicoes_Todas")
        top_suspeitos.to_excel(wr, index=False, sheet_name="Top_Suspeitos")
        # Métricas hold-out (se houver)
        if y_test is not None and y_pred is not None:
            cm = confusion_matrix(y_test, y_pred)
            cm_df = pd.DataFrame(cm, index=["Neg","Pos"], columns=["Pred_Neg","Pred_Pos"])
            cm_df.to_excel(wr, sheet_name="Matriz_Confusao")
            pd.DataFrame([{"Accuracy":acc,"F1":f1,"ROC_AUC":auc}]).to_excel(wr, index=False, sheet_name="Metricas")

    print("OK: ML_predictions.xlsx (CV + predições + suspeitos + métricas) e modelos .pkl salvos em OUT.")
else:
    # Mesmo sem treino, registre a comparação CV
    with pd.ExcelWriter(str(Path(outdir)/"ML_predictions.xlsx"), engine="xlsxwriter") as wr:
        df_resultados.to_excel(wr, index=False, sheet_name="Comparacao_CV")
    print("Aviso: ML sem hold-out (dados insuficientes). Exportado apenas Comparacao_CV.")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

RF (hold-out) -> ACC: 0.7929 | F1: 0.7752 | ROC_AUC: 0.8901
OK: ML_predictions.xlsx (CV + predições + suspeitos + métricas) e modelos .pkl salvos em OUT.


14) Gerar arquivo ZIP com as análises realizadas.


In [18]:
# Gera um ZIP com tudo que está em /content/OUT e baixa
import shutil
from google.colab import files
zip_path = "/content/relatorios_out"
shutil.make_archive(zip_path, "zip", "/content/OUT")
files.download(zip_path + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>